In [37]:
# load libraries
import google.datalab.bigquery as bq
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt

In [70]:
%%bq query 

SELECT COUNT(DISTINCT emp_id) as num_employees
FROM `w210-fedwork.processed_data.status`


num_employees
4375415


In [1]:
%%bq query 

SELECT COUNT(DISTINCT emp_id) as num_employees
FROM `w210-fedwork.processed_data.dynamic`

num_employees
3871680


#### Are there employees in dynamic that do NOT exist in status?

In [71]:
%%bq query 

SELECT COUNT(DISTINCT a.emp_id) as num_employees, COUNT(*) as num_records
FROM `w210-fedwork.processed_data.dynamic` as a 
LEFT OUTER JOIN `w210-fedwork.processed_data.status` as b
ON a.emp_id=b.emp_id
WHERE b.emp_id IS NULL


num_employees,num_records
150475,267640


###### Should we join on agency? Check the number of employees with multiple agencies in status

In [72]:
%%bq query 
SELECT count(emp_id) as num_emp_mult_agencies
FROM (
    SELECT emp_id, year_file_dt, quarter_file_dt, COUNT(DISTINCT agency) as num_agencies
    FROM `w210-fedwork.processed_data.status`
    GROUP BY emp_id,year_file_dt, quarter_file_dt
  )
WHERE num_agencies >1


num_emp_mult_agencies
157658


#### Join status and dynamic. 


##### Number employees in joint table (merging on employee id only)

In [76]:
%%bq query 

-- First, join on emp

SELECT COUNT(DISTINCT s.emp_id) as num_emp, COUNT(*) as num_records
FROM `w210-fedwork.processed_data.status` as s
JOIN `w210-fedwork.processed_data.dynamic` as d
ON s.emp_id=d.emp_id 


num_emp,num_records
3721205,150544943


##### Number employees in joint table (merging on employee id, year and quarter)

In [74]:
%%bq query 

-- Second, join on emp, year, quarter
SELECT COUNT(DISTINCT s.emp_id) as num_emp, COUNT(*) as num_records
FROM `w210-fedwork.processed_data.status` as s
JOIN `w210-fedwork.processed_data.dynamic` as d
ON s.emp_id=d.emp_id 
AND s.year_file_dt=d.year_eff_dt 
AND s.quarter_file_dt=d.quarter_eff_dt 



num_emp,num_records
2858649,4741798


##### Number employees in joint table (merging on employee id, year and quarter, agency)

In [75]:
%%bq query 

-- Third, join on emp, year, quarter, agency
SELECT COUNT(DISTINCT s.emp_id) as num_emp, COUNT(*) as num_records
FROM `w210-fedwork.processed_data.status` as s
JOIN `w210-fedwork.processed_data.dynamic` as d
ON s.emp_id=d.emp_id 
AND s.year_file_dt=d.year_eff_dt 
AND s.quarter_file_dt=d.quarter_eff_dt 
AND s.agency=d.agency 

num_emp,num_records
2852560,4183605


### Analysis of table status_dynamic



* Note, the merge to create table status_dynamic was done with keeping all records from status and records from dynamic that match on emp_id, year, quarter and agency

##### Num employees in status_dynamic

In [60]:
%%bq query 

SELECT COUNT(DISTINCT emp_id) as num_emp
FROM `w210-fedwork.processed_data.status_dynamic`

num_emp
4375415


##### Num employees in status_dynamic with records excluded from dynamic table

In [59]:
%%bq query 

SELECT COUNT(DISTINCT emp_id) as num_emp_records_excluded

FROM (

  --Get max year-quarter from merged table and max-year quarter from records excluded from the merge

  SELECT emp_id, max(year) as year, max(quarter) as quarter, max(year_eff_dt) as year_ex, max(quarter_eff_dt) as quarter_ex
  FROM (

      --Get employees who exists in both dynamic and status tables and who have records excluded from the merge on emp_id, year and quarter. 
      SELECT a1.emp_id, a1.year, a1.quarter, ex.year_eff_dt, ex.quarter_eff_dt
      FROM `w210-fedwork.processed_data.status_dynamic` as a1
      JOIN (

          -- Get records that are excluded from the merge on emp_id, year, quarter (for now we will ignore agency)
          SELECT DISTINCT a.*
          FROM `w210-fedwork.processed_data.dynamic` as a
          LEFT OUTER JOIN `w210-fedwork.processed_data.status_dynamic` as b
          ON a.emp_id=b.emp_id 
          AND a.year_eff_dt=b.year 
          AND a.quarter_eff_dt=b.quarter
          WHERE b.emp_id IS NULL
      ) AS ex
      ON a1.emp_id=ex.emp_id

  )
  GROUP BY emp_id
)

num_emp_records_excluded
2033471


##### Num emp in status_dynamic table with records excluded from dynamic where file date > effective date (using year and quarter for both dates)


In [62]:
%%bq query 

SELECT COUNT(DISTINCT emp_id) as num_emp_records_exc_dyn_gt_merge

FROM (

  --Get max year-quarter from merged table and max-year quarter from records excluded from the merge

  SELECT emp_id, max(year) as year, max(quarter) as quarter, max(year_eff_dt) as year_dynamic, max(quarter_eff_dt) as quarter_dynamic
  FROM (

      --Get employees who had record excluded from the merge of dynamic and status tables. 
      SELECT a1.emp_id, a1.year, a1.quarter, ex.year_eff_dt, ex.quarter_eff_dt
      FROM `w210-fedwork.processed_data.status_dynamic` as a1
      JOIN (

          -- Get records that were excluded from the merge on emp_id, year, quarter (for now we will ignore agency)
          SELECT DISTINCT a.*
          FROM `w210-fedwork.processed_data.dynamic` as a
          LEFT OUTER JOIN `w210-fedwork.processed_data.status_dynamic` as b
          ON a.emp_id=b.emp_id 
          AND a.year_eff_dt=b.year 
          AND a.quarter_eff_dt=b.quarter
          WHERE b.emp_id IS NULL
      ) AS ex
      ON a1.emp_id=ex.emp_id

  )
  GROUP BY emp_id
)  
WHERE (year>year_dynamic) or (year=year_dynamic and quarter>quarter_dynamic)


num_emp_records_exc_dyn_gt_merge
996589


##### Num emp in status_dynamic table with records excluded from dynamic where file date < effective date (using year and quarter for both dates)

* These might be records where accessing indicator is quit, retirement, etc
* These might be records where the record for quit or retirement was file in the following quarter

In [63]:
%%bq query 
SELECT COUNT(DISTINCT emp_id) as num_emp_records_exc_dyn_lt_merge

FROM (

  --Get max year-quarter from merged table and max-year quarter from records excluded from the merge

  SELECT emp_id, max(year) as year, max(quarter) as quarter, max(year_eff_dt) as year_ex, max(quarter_eff_dt) as quarter_ex
  FROM (

      --Get employees who had record excluded from the merge of dynamic and status tables. 
      SELECT a1.emp_id, a1.year, a1.quarter, ex.year_eff_dt, ex.quarter_eff_dt
      FROM `w210-fedwork.processed_data.status_dynamic` as a1
      JOIN (

          -- Get records that were excluded from the merge on emp_id, year, quarter (for now we will ignore agency)
          SELECT DISTINCT a.*
          FROM `w210-fedwork.processed_data.dynamic` as a
          LEFT OUTER JOIN `w210-fedwork.processed_data.status_dynamic` as b
          ON a.emp_id=b.emp_id 
          AND a.year_eff_dt=b.year 
          AND a.quarter_eff_dt=b.quarter
          WHERE b.emp_id IS NULL
      ) AS ex
      ON a1.emp_id=ex.emp_id

  )
  GROUP BY emp_id
)  
WHERE (year<year_ex) or (year=year_ex and quarter<quarter_ex)

num_emp_records_exc_dyn_lt_merge
791343


##### Example of employees with records excluded from dynamic table where file date (status) < effective date (dynamic) 

In [2]:
%%bq query 

SELECT emp_id, year, quarter, year_dynamic, quarter_dynamic

FROM (

  --Get max year-quarter from merged table and max-year quarter from records excluded from the merge

  SELECT emp_id, max(year) as year, max(quarter) as quarter, max(year_eff_dt) as year_dynamic, max(quarter_eff_dt) as quarter_dynamic
  FROM (

      --Get employees who had record excluded from the merge of dynamic and status tables. 
      SELECT a1.emp_id, a1.year, a1.quarter, ex.year_eff_dt, ex.quarter_eff_dt
      FROM `w210-fedwork.processed_data.status_dynamic` as a1
      JOIN (

          -- Get records that were excluded from the merge on emp_id, year, quarter (for now we will ignore agency)
          SELECT DISTINCT a.*
          FROM `w210-fedwork.processed_data.dynamic` as a
          LEFT OUTER JOIN `w210-fedwork.processed_data.status_dynamic` as b
          ON a.emp_id=b.emp_id 
          AND a.year_eff_dt=b.year 
          AND a.quarter_eff_dt=b.quarter
          WHERE b.emp_id IS NULL
      ) AS ex
      ON a1.emp_id=ex.emp_id

  )
  GROUP BY emp_id
)  
WHERE (year<year_dynamic) or (year=year_dynamic and quarter<quarter_dynamic)
LIMIT 15

emp_id,year,quarter,year_dynamic,quarter_dynamic
006455815,2003,4,2004,1
002413311,2003,4,2004,1
001309278,2004,4,2005,1
008397061,2005,4,2006,1
009093726,2005,4,2006,1
001258782,2006,4,2007,1
000942535,2006,4,2007,1
002142253,2007,4,2008,1
000126425,2007,4,2008,1
005766556,2008,4,2009,4


In [4]:
%%bq query 
select *
FROM `w210-fedwork.processed_data.dynamic`
where emp_id in ('002646424', '003128136','005766556')
order by emp_id, effective_dt

emp_id,agency,acces_sep_ind,effective_dt,age,pay_plan,grade,los,duty_station,occupation,occupation_cat,basic_pay,type_appoint,work_schedule,dod_ind,year_eff_dt,month_eff_dt,day_eff_dt,quarter_eff_dt,week_eff_dt
002646424,VATA,AC,2000-12-03,45-49,WG,01,UNSP,#########,3566,B,15840,15,F,0,2000,12,3,4,49
002646424,VATA,SC,2001-01-05,40-44,WG,01,< 1,#########,3566,B,15840,15,F,0,2001,1,5,1,0
002646424,TD19,AD,2002-08-11,45-49,SV,FF,UNSP,#########,0019,T,33787,38,F,0,2002,8,11,3,32
002646424,TD19,SB,2003-03-08,45-49,SV,FF,< 1,#########,0019,T,35149,38,F,0,2003,3,8,1,9
002646424,HSBC,AB,2003-03-09,45-49,SV,FF,< 1,#########,0019,T,35149,38,F,0,2003,3,9,1,10
003128136,DJ09,SD,2012-03-24,55-59,AD,31,20-24,#########,0905,P,154352,38,F,0,2012,3,24,1,12
005766556,DJ09,AD,2002-12-29,30-34,AD,24,UNSP,#########,0905,P,72768,48,F,0,2002,12,29,4,52
005766556,DJ01,SA,2007-01-03,35-39,GS,15,5-9,110010001,0905,P,118272,44,F,0,2007,1,3,1,0
005766556,NS00,AD,2007-10-07,35-39,SL,00,5-9,110010001,0301,A,132437,40,F,0,2007,10,7,4,40
005766556,NS00,SC,2009-01-20,35-39,SL,00,5-9,110010001,0301,A,138380,40,F,0,2009,1,20,1,3


In [5]:
%%bq query 
select *
FROM `w210-fedwork.processed_data.status`
where emp_id in ('002646424', '003128136','005766556')
order by emp_id, file_dt

emp_id,file_dt,agency,duty_station,age,education_level,pay_plan,grade,los,occupation,occupation_cat,basic_pay,superv_status,type_appoint,work_schedule,nsftp_ind,dod_ind,year_file_dt,month_file_dt,day_file_dt,quarter_file_dt,week_file_dt
002646424,2000-12-13,VATA,#########,45-49,11,WG,01,UNSP,3566,B,15840,8,15,F,1,0,2000,12,13,4,50
002646424,2002-09-30,TD19,#########,45-49,04,SV,FF,< 1,0019,T,33787,8,38,F,1,0,2002,9,30,3,39
002646424,2002-12-13,TD19,#########,45-49,04,SV,FF,< 1,0019,T,33787,8,38,F,1,0,2002,12,13,4,49
002646424,2003-03-31,HSBC,#########,45-49,04,SV,FF,< 1,0019,T,35149,8,38,F,1,0,2003,3,31,1,13
002646424,2003-06-30,HSBC,#########,45-49,04,SV,FF,< 1,0019,T,35149,8,38,F,1,0,2003,6,30,2,26
002646424,2003-09-30,HSBC,#########,45-49,04,SV,FF,1-2,0019,T,35149,8,38,F,1,0,2003,9,30,3,39
002646424,2003-12-13,HSBC,#########,50-54,04,SV,FF,1-2,0019,T,35149,8,38,F,1,0,2003,12,13,4,49
002646424,2004-03-31,HSBC,#########,45-49,04,SV,FF,1-2,0019,T,36697,8,38,P,2,0,2004,3,31,1,13
002646424,2004-06-30,HSBC,#########,50-54,04,SV,FF,1-2,0019,T,36697,8,38,P,2,0,2004,6,30,2,26
002646424,2004-09-30,HSBC,#########,50-54,04,SV,FF,1-2,0019,T,36697,8,38,P,2,0,2004,9,30,3,39


##### Num employees with multiple work status (partime and full time)

In [77]:
%%bq query 

SELECT COUNT(DISTINCT emp_id) as num_emp_multiple_status
FROM (
   SELECT emp_id, max(nsftp_ind) as max_nsftp, min(nsftp_ind) as min_nsftp
   FROM `w210-fedwork.processed_data.status_dynamic` 
   GROUP BY emp_id
   HAVING max_nsftp<>min_nsftp
   

)

num_emp_multiple_status
639265


In [10]:
%%bq query 

SELECT COUNT(DISTINCT emp_id) as num_emp_one_month_excluded
--emp_id, year, quarter, file_dt, year_dynamic, quarter_dynamic, effective_dt

FROM (

  --Get max year-quarter from merged table and max-year quarter from records excluded from the merge

  SELECT emp_id, max(year) as year, max(quarter) as quarter, max(file_dt) as file_dt, max(year_eff_dt) as year_dynamic, max(quarter_eff_dt) as quarter_dynamic, max(effective_dt) as effective_dt
  FROM (

      --Get employees who had records excluded from the merge of dynamic and status tables. 
      SELECT a1.emp_id, a1.year, a1.quarter, a1.file_dt, ex.year_eff_dt, ex.quarter_eff_dt, ex.effective_dt
      FROM `w210-fedwork.processed_data.status_dynamic` as a1
      JOIN (

          -- Get records that were excluded from the merge on emp_id, year, quarter (for now we will ignore agency)
          SELECT DISTINCT a.*
          FROM `w210-fedwork.processed_data.dynamic` as a
          LEFT OUTER JOIN `w210-fedwork.processed_data.status_dynamic` as b
          ON a.emp_id=b.emp_id 
          AND a.year_eff_dt=b.year 
          AND a.quarter_eff_dt=b.quarter
          WHERE b.emp_id IS NULL
      ) AS ex
      ON a1.emp_id=ex.emp_id
  )
  GROUP BY emp_id
)  
WHERE ((year<year_dynamic) or (year=year_dynamic and quarter<quarter_dynamic))
AND (DATE_DIFF(effective_dt,file_dt, MONTH)=1)  
--LIMIT 15



num_emp_one_month_excluded
307131


Summary

* Num employees in status table : 4,375,415 
* Num employees in dynamic table : 3,871,680 
* Num employees in dynamic that do NOT exist in status table : 150,475 
* Num employees in status with multiple agencies :  157,658   (3.6%) We can join on agency since we will not be dropping many records. 
* Num employees in merged table 'status_dynamic' that have records excluded from the merge on year, quarter : 2,033,471 


status_dynamic table (created with all records from status and using values from dynamic for those records that join on emp_id, year, quarter, agency)

* Num employees in table 'status_dynamic' : 4,375,415. 
* Num employees in merged table 'status_dynamic' that have records excluded where file_date (status table) > effective_date (dynamic table) : 996,589  . Expected, people have not quit, or have not had a change. 
* Num employees in merged table 'status_dynamic' that have records excluded where file_date (status table) < effective_date (dynamic table) : 791,343  . See example below
* Num employees in merged table 'status_dynamic' that have records excluded with one month appart between file_date and effective_date: 307,131
* Num employees in merged table 'status_dynamic' that have records as full time and part time employees (nsftp=1 and 2) : 639,265



Analysis of possible joins

* Num employees that exist in status and dynamic tables when joining on emp_id : 3,721,205 (total number of merged records: 150,544,943) 
* Num employees that exist in status and dynamic tables when joining on emp_id, year, quarter : 2,858,649 (total number of merged records: 4,741,798) 
* Num employees that exist in status and dynamic tables when joining on emp_id, year, quarter, agency : 2,852,560 (total number of merged records: 4,183,605) 


Comments

* It is safe to include agency in the join since joining on agency does not significantly reduce the number of records.



### Additional EDA for invalid values in status_dyanmic table

In [20]:
%%bq query 

SELECT duty_station, COUNT(*) as num_records
FROM `w210-fedwork.processed_data.status_dynamic`
WHERE duty_station IN ('*********', '#########', '         ')
GROUP BY duty_station


duty_station,num_records
#########,61019797
*********,61356


In [21]:
%%bq query 

SELECT education_level, COUNT(*) as num_records
FROM `w210-fedwork.processed_data.status_dynamic`
WHERE (education_level IN ('**', '##', '')) 
GROUP BY education_level

education_level,num_records
,2510987
**,121605


In [7]:
%%bq query 

SELECT grade, COUNT(*) as num_records
FROM `w210-fedwork.processed_data.status`
WHERE (grade IN ('**', '##', '')) 
GROUP BY grade

grade,num_records
,8921
**,5961


In [8]:
%%bq query 

SELECT occupation, COUNT(*) as num_records
FROM `w210-fedwork.processed_data.status`
WHERE (occupation IN ('****', '####', '')) 
GROUP BY occupation

occupation,num_records
****,23981


In [9]:
%%bq query 

SELECT occupation_cat, COUNT(*) as num_records
FROM `w210-fedwork.processed_data.status`
WHERE (occupation_cat IN ('*', '#', '')) 
GROUP BY occupation_cat

occupation_cat,num_records
*,25887


In [10]:
%%bq query 

SELECT pay_plan, COUNT(*) as num_records
FROM `w210-fedwork.processed_data.status`
WHERE (pay_plan IN ('**', '##', '')) 
GROUP BY pay_plan

pay_plan,num_records
**,5099


In [11]:
%%bq query 

SELECT superv_status, COUNT(*) as num_records
FROM `w210-fedwork.processed_data.status`
WHERE (superv_status IN ('*', '#', '', ' ')) 
GROUP BY superv_status

superv_status,num_records
*,10573


In [12]:
%%bq query 

SELECT type_appoint, COUNT(*) as num_records
FROM `w210-fedwork.processed_data.status`
WHERE (type_appoint IN ('**', '##', '', '  ')) 
GROUP BY type_appoint

type_appoint,num_records
**,14057


In [14]:
%%bq query 

SELECT work_schedule, COUNT(*) as num_records
FROM `w210-fedwork.processed_data.status`
WHERE (work_schedule IN ('*', '#', '', ' ')) 
GROUP BY work_schedule

work_schedule,num_records
*,1556


In [15]:
%%bq query 

SELECT duty_station, COUNT(*) as num_records
FROM `w210-fedwork.processed_data.dynamic`
WHERE (work_schedule IN ('*********', '#########', '', '         ')) 
GROUP BY duty_station

duty_station,num_records
*********,5
#########,15
110010001,4
RQ0450000,2
480490245,1
426540101,1
251430013,1
282455151,1


In [16]:
%%bq query 

SELECT grade, COUNT(*) as num_records
FROM `w210-fedwork.processed_data.dynamic`
WHERE (grade IN ('**', '##', '')) 
GROUP BY grade

grade,num_records
**,3795
,35


### Ignore the rest for now

In [32]:
emp_comb = bq.Query('SELECT emp_id, file_dt, agency, duty_station, age, education_level, pay_plan, grade, los \
                              ,occupation, occupation_cat, superv_status, type_appoint, work_schedule \
                             ,nsftp_ind, acces_sep_ind \
                        FROM `processed_data.status_dynamic` \
                        LIMIT 100000')
emp_comb = emp_comb.execute(output_options=bq.QueryOutput.dataframe()).result()
emp_comb.head()

,emp_id,file_dt,agency,duty_station,age,education_level,pay_plan,grade,los,occupation,occupation_cat,superv_status,type_appoint,work_schedule,nsftp_ind,acces_sep_ind
0,010089894,2005-09-30,LT00,110010001,20-24,08,None,None,5-9,None,None,8,10,F,1,SC
1,004965299,2003-09-30,HE37,462800023,20-24,01,None,None,1-2,None,None,8,48,F,2,None
2,009948076,2003-09-30,LT00,*********,UNSP,None,GS,None,UNSP,None,None,None,None,None,2,None
3,001294448,2009-06-30,HSBD,#########,60-64,10,GS,None,25-29,None,None,8,10,F,1,None
4,001981593,2002-06-30,ARX6,*********,50-54,04,None,None,UNSP,None,None,8,10,None,2,None


In [ ]:
# emp_comb = bq.Query.from_table(bq.Table('processed_data.status_dynamic'), fields=['emp_id', 'file_dt', 'agency', 'duty_station', 'age', 'education_level', 'pay_plan', 'grade', 'los'
#                                                                             ,'occupation', 'occupation_cat', 'basic_pay', 'superv_status', 'type_appoint', 'work_schedule'
#                                                                             , 'nsftp_ind', 'acces_sep_ind']).execute().result().to_dataframe().set_index('emp_id', 'file_dt','agency')

In [33]:
emp_comb.describe()

,emp_id,file_dt,agency,duty_station,age,education_level,pay_plan,grade,los,occupation,occupation_cat,superv_status,type_appoint,work_schedule,nsftp_ind,acces_sep_ind
count,100000,100000,100000,100000,100000,99326,99919,99887,100000,99982,99959,99999,99993,99998,100000,4472
unique,75089,24,91,1005,14,22,35,161,11,246,6,6,14,6,2,13
top,003237193,2002-12-13,TD03,#########,45-49,04,SV,DD,5-9,2152,A,8,38,F,1,AD
freq,7,6052,42591,52431,17352,38356,29369,8591,17031,22400,52082,77880,75549,88081,83917,3677


In [34]:
import seaborn as sns

In [54]:
data_dummies = pd.get_dummies(emp_comb, columns =['age', 'education_level', 'occupation_cat', 'type_appoint', 'work_schedule', 'acces_sep_ind'])

In [50]:
data_dummies.head()

,emp_id,file_dt,agency,duty_station,los,occupation,superv_status,nsftp_ind,age_15-19,age_20-24,...,acces_sep_ind_SA,acces_sep_ind_SC,acces_sep_ind_SD,acces_sep_ind_SE,acces_sep_ind_SF,acces_sep_ind_SG,acces_sep_ind_SH,acces_sep_ind_SJ,acces_sep_ind_SK,acces_sep_ind_SL
0,010089894,2005-09-30,LT00,110010001,5-9,None,8,1,0,1,...,0,1,0,0,0,0,0,0,0,0
1,004965299,2003-09-30,HE37,462800023,1-2,None,8,2,0,1,...,0,0,0,0,0,0,0,0,0,0
2,009948076,2003-09-30,LT00,*********,UNSP,None,None,2,0,0,...,0,0,0,0,0,0,0,0,0,0
3,001294448,2009-06-30,HSBD,#########,25-29,None,8,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,001981593,2002-06-30,ARX6,*********,UNSP,None,8,2,0,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
data_dummies.columns

Index([u'emp_id', u'file_dt', u'agency', u'duty_station', u'pay_plan',
       u'grade', u'los', u'occupation', u'superv_status', u'nsftp_ind',
       u'age_15-19', u'age_20-24', u'age_25-29', u'age_30-34', u'age_35-39',
       u'age_40-44', u'age_45-49', u'age_50-54', u'age_55-59', u'age_60-64',
       u'age_65-69', u'age_70-74', u'age_75+', u'age_UNSP',
       u'education_level_01', u'education_level_02', u'education_level_03',
       u'education_level_04', u'education_level_05', u'education_level_06',
       u'education_level_07', u'education_level_08', u'education_level_09',
       u'education_level_10', u'education_level_11', u'education_level_12',
       u'education_level_13', u'education_level_14', u'education_level_15',
       u'education_level_16', u'education_level_17', u'education_level_18',
       u'education_level_19', u'education_level_20', u'education_level_21',
       u'education_level_22', u'occupation_cat_A', u'occupation_cat_B',
       u'occupation_cat_C', u'occupatio